In [1]:
import re
import pandas as pd
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions, calculate_pnl, process_and_save_address
'''
data_process.ipynb和data_fetch.ipynb同样功能，但是通过封装函数实现了给日期地址等参数就可以返回数据，更加简洁
添加了pnl的计算，我们这里只计算第一种交易的pnl，其余的暂时忽略不计。
最后对于address也进行一个处理
'''
# ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
ADDRESS = "0x2c89a2ee92b9870f55989b4132a58c0e85222d86"
# ADDRESS = "0x6c2a355929ee1262305e385ad49b84fe5f5a4777"

START_DATE = "2024-01-01 08:00" # Singapore Time，the function will convert it to UTC
END_DATE = "2025-01-01 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE", "DAI"}
base_pattern = re.compile('|'.join(BASE_TOKENS))

In [3]:
transfers_df = fetch_and_save_erc20_transfers(
        address=ADDRESS,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE,
        offset=1000,
    )
transfers_df = pd.read_csv(OUTPUT_FILE + "/erc20_transfers.csv")
record_df_1, record_df_2, record_df_3 = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, base_pattern)
# 这里我为了显示简洁把process_transactions中print结果的注释掉了 如果需要可以取消注释
if not record_df_1.empty:
    unique_tokens_count = record_df_1["token_symbol"].nunique()
    unique_tokens = record_df_1["token_symbol"].unique()
    results = []

    for token in unique_tokens:
        token_df = record_df_1[record_df_1["formatted_record"].str.contains(fr"\b{token}\b", regex=True)]
        price_unit = token_df["price_unit"].iloc[0]
        print(f"{token} ({price_unit}): {len(token_df)} records")

        pnl_df = calculate_pnl(token_df)
        print(tabulate(pnl_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))   
        total_pnl = pnl_df["pnl"].sum()

        results.append({"token": token, "price_unit": price_unit, "total_pnl": total_pnl})

    pnl_summary_df = pd.DataFrame(results)

    print(tabulate(pnl_summary_df, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
else:
    print("No transactions found")


2025-01-14 15:34:09,528 - INFO - Start block: 18908894, End block: 21525891
2025-01-14 15:34:09,529 - INFO - Fetching ERC20 transfers from block 18908894 to 21525891...
2025-01-14 15:34:12,154 - INFO - Page 1: Retrieved 1000 transactions.
2025-01-14 15:34:13,927 - INFO - Page 2: Retrieved 543 transactions.
2025-01-14 15:34:14,861 - WARNING - API request failed on page 3: Request failed: No transactions found
2025-01-14 15:34:14,865 - INFO - Final dataset contains 1543 unique transactions.
2025-01-14 15:34:14,890 - INFO - Data successfully saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result\erc20_transfers.csv
2025-01-14 15:34:14,890 - INFO - Transfers saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result


...Merging 456 groups of transactions...
Transaction Hash 0x0fcc53040dfc925d1b169fbedc575f9eec30fad4133bdd38ed8efdfc3f2969d3 contains 6 records:
Transaction Hash 0x1f4ca78710717a409b2cee42702db582b13d4e544adf116a86c605f92973d935 contains 6 records:
Transaction Hash 0x2b198f8b99cc4a183fdb5125b09e1f32521b7f5351e6b12307d5022ba030708c contains 4 records:
Transaction Hash 0x3de619b29ddab8d241f47d7a21bde8318691b71244e4658dedf97f5eba4705b3 contains 4 records:
Transaction Hash 0x3f1e3bfa355baded87207a2de30165c09a64d1ecb070f04c819587cc9386a1b0 contains 4 records:
Transaction Hash 0x3f550a50553c3033b0b0d32a435d1c0a9d3ba8323a4f9a8de1ffdcaa2cbaf739 contains 20 records:
Transaction Hash 0x427a515c5b0967335ebc0dc437bdc75ca1d2cf6d5573a6e477b399de40dcb9a6 contains 4 records:
Transaction Hash 0x52dd542f161f822026ef0050e6002c161957d086113dd7e5bf23a6e54ee02074 contains 6 records:
Transaction Hash 0x53d7b42ca96eaa61f5fd8e49624078d39c26286ef30bc5223bfdcf2ca2856a9e contains 4 records:
Transaction Hash 0x564

In [5]:
TOP_ACCOUNTS_DF = pd.read_csv(OUTPUT_FILE + "/top_accounts.csv")
transfers_df = pd.read_csv(OUTPUT_FILE + "/erc20_transfers.csv")
output_file = OUTPUT_FILE + "/address_counts_with_name_tags.csv"

process_and_save_address(transfers_df, TOP_ACCOUNTS_DF, output_file)

100%|██████████| 44/44 [01:20<00:00,  1.84s/it]

Data has been save to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result/address_counts_with_name_tags.csv


In [4]:
address_counts = pd.read_csv(OUTPUT_FILE + "/address_counts_with_name_tags.csv")
address_threshold = 10
important_addresses = address_counts[address_counts["total_value_count"] > address_threshold]
important_addresses = important_addresses.sort_values(by="total_value_count", ascending=False)
# important_addresses = important_addresses[important_addresses["address"] != ADDRESS.lower()]

print(important_addresses)


                                       address  from_count  to_count  \
11  0x5be9a4959308a0d0c7bc0870e319314d8d957dbb         112      4151   
31  0xe217e15b3c19cc0427f9492dc3bcfe8220afad10        4097         1   
20  0x9008d19f58aabd9ed0d60971565aa8510560ab41          40        73   

    total_value_count  is_in_top_accounts                      name_tag  
11               4263               False       World Liberty: Multisig  
31               4098               False   TransparentUpgradeableProxy  
20                113               False  CoW Protocol: GPv2Settlement  
